In [9]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils
from redcap_funcs import comp_schema_cap_db, get_project_schema

db_name = 'CAPTIVA'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'CAPTIVA Data Collection'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name) 

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

In [28]:
crf_name = 'baseline_brain_crf'

api_questions = pd.DataFrame(project.export_metadata())
api_questions = api_questions[api_questions['form_name'] == crf_name]
field_names = pd.DataFrame(project.export_field_names())
field_names.rename(columns={'original_field_name': 'field_name'}, inplace=True)
# api_questions = pd.merge(field_names, api_questions, on='field_name')
api_questions = pd.merge(api_questions, field_names, on='field_name', how='left')


# replacing
def val_to_text(row):
    if row['field_type'] == 'checkbox':
        dic = {op.split(',')[0].strip() : op.split(',')[1].strip() for op in row['select_choices_or_calculations'].split('|')}
        return dic[row['choice_value']]
    else:
        return row['select_choices_or_calculations']
api_questions['select_choices_or_calculations'] = api_questions.apply(val_to_text, axis=1)

def replace_seperators(row):
    if row['field_type'] == 'radio':
        return row['select_choices_or_calculations'].replace(',', '=')
    else:
        return row['select_choices_or_calculations']
api_questions['select_choices_or_calculations'] = api_questions.apply(replace_seperators, axis=1)

api_questions.loc[(api_questions['field_type'] == 'checkbox') | 
                (api_questions['field_type'] == 'radio') | 
                (api_questions['field_type'] == 'yesno'), 'data_type'] = 'int'
api_questions.loc[api_questions['field_type'] == 'text', 'data_type'] = 'string'

api_questions.loc[api_questions['export_field_name'].str.contains('___', na=False), 'export_field_name'] = api_questions['export_field_name'].apply(lambda x: 
                                                                                                            x.split('___')[0] if isinstance(x, str) else x)+'('+api_questions['choice_value']+')'
api_questions['redcap_variable'] = api_questions['export_field_name']
api_questions

,field_name,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,...,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation,choice_value,export_field_name,data_type
0,baseline_brain_crf_query,baseline_brain_crf,,notes,Query:,,,,,,...,,,,,,,@READONLY\t\r\n@SETVALUE='[baseline_brain_crf_...,,baseline_brain_crf_query,NaN
1,q1001,baseline_brain_crf,,radio,Q1001. Scan type,1= CT | 2= MR | 3= CT and MR,,,,,...,,,,,,,,,q1001,int
2,q1001a,baseline_brain_crf,,radio,Q1001a. Scan done with contrast?,0= No | 1= Yes,,,,,...,,,,,,,@DEFAULT='0',,q1001a,int
3,q1002,baseline_brain_crf,,text,Q1002. Date of imaging,,,date_mdy,,,...,,,,,,,,,q1002,string
4,q1003,baseline_brain_crf,,text,Q1003. Time of imaging,,,time,,,...,,,,,,,,,q1003,string
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,q1014,baseline_brain_crf,,text,Q1014. Initials of Central Reader,,,,,,...,,,,,,,,,q1014,string
78,bl_brain_qc,baseline_brain_crf,,text,QC. Date of Central Reader's Initials,,,date_mdy,,,...,,,,,,,,,bl_brain_qc,string
79,baseline_brain_crf_status,baseline_brain_crf,"<div class=""rich-text-field-label""><p>Lab use ...",radio,Baseline: CRF status,1= Unread | 2= Query generated | 3= Read - Unv...,,,,,...,,,,,,,@IF([user-role-label] = 'Admin' OR [user-role-...,,baseline_brain_crf_status,int
80,baseline_brain_crf_comments,baseline_brain_crf,,notes,Baseline: CRP Comments,,,,,,...,,,,,,,@IF([user-role-label] = 'Admin' OR [user-role-...,,baseline_brain_crf_comments,NaN


In [ ]:
s = 'dfsdf___sdfs'
s.split('___')

In [18]:
comp_schema_cap_db(db_name, project_name)

/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')



-----------baseline_brain_crf-----------

The following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:
{redcap_variable : question_text}

{'q1011(hi1)': 'Q1011. Acute hemorrhage classification (Check all that are applicable)', 'q1011(hi2)': 'Q1011. Acute hemorrhage classification (Check all that are applicable)', 'q1011(ph1)': 'Q1011. Acute hemorrhage classification (Check all that are applicable)', 'q1011(ph2)': 'Q1011. Acute hemorrhage classification (Check all that are applicable)'}


---------baseline_vascular_crf----------

The following CRF_Data_RedCap.redcap_variable's are not in CRF_schema_RedCap.redcap_variable's:
['baseline_vascular_crf_complete', 'q2031', 'q2032', 'q2033', 'q2034', 'q2035', 'q2036', 'q2037', 'q2038', "'SYMPTOMATIC VESSEL IS RIGHT M2\r SUPERIOR DIVISION DS", "'SYMPTOMATIC VESSEL IS RIGHT M2 SUPERIOR DIVISION DS", '0.3; DN', "'SYMPTOMATIC VESSEL LEFT P1 WITH DS", '0.1 AND DN']


----------follow_up_brain_crf-----------

The followi

Exception: Please handle the above discrepancies